# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
#Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [====================

Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4166 - mse: 0.4166 - val_loss: 0.1954 - val_mse: 0.1954
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.1779 - val_mse: 0.1779
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1746 - mse: 0.1746 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1504 - mse: 0.1504 - val_loss: 0.1443 - val_mse: 0.1443
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1374 - mse: 0.1374 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1246 - mse: 0.1246 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1141 - mse: 0.1141 - val_loss: 0.1444 - val_mse: 0.1444
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1077 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1419 - val_mse: 0.1419
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.1440 - val_mse: 0.1440
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1433 - val_mse: 0.1433
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1506 - val_mse: 0.1506
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1491 - val_mse: 0.1491
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1517 - val_mse: 0.1517
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1508 - val_mse: 0.1508
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1520 - val_mse: 0.1520
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0073 - mse: 0.0073


[0.007314360700547695, 0.007314360700547695]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 984us/step - loss: 0.1524 - mse: 0.1524


[0.15235896408557892, 0.15235896408557892]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

30673.652050711094

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4813 - mse: 0.4813 - val_loss: 0.1674 - val_mse: 0.1674
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2307 - mse: 0.2307 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1927 - mse: 0.1927 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1682 - mse: 0.1682 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1532 - mse: 0.1532 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1492 - mse: 0.1492 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1255 - mse: 0.1255 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1207 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1327 - val_mse: 0.1327
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1355 - val_mse: 0.1355
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1356 - val_mse: 0.1356
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0224 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1443 - val_mse: 0.1443
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1462 - val_mse: 0.1462
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0089 - mse: 0.0089


[0.00888123083859682, 0.00888123083859682]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 911us/step - loss: 0.1469 - mse: 0.1469


[0.14689449965953827, 0.14689449965953827]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.4254 - mse: 0.4254 - val_loss: 0.1762 - val_mse: 0.1762
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2198 - mse: 0.2198 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1758 - mse: 0.1758 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1619 - mse: 0.1619 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1319 - mse: 0.1319 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1166 - mse: 0.1166 - val_loss: 0.1229 - val_mse: 0.1229
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1153 - mse: 0.1153 - val_loss: 0.1422 - val_mse: 0.1422
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0992 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0191 - mse: 0.0191 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.1057 - val_mse: 0.1057
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0183 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1052 - val_mse: 0.1052
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0062 - mse: 0.0062


[0.006204873789101839, 0.006204873789101839]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 900us/step - loss: 0.1076 - mse: 0.1076


[0.10764630883932114, 0.10764630883932114]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3165 - mse: 0.3165 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1564 - mse: 0.1564 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1109 - mse: 0.1109 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0888 - mse: 0.0888 - val_loss: 0.1133 - val_mse: 0.1133
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0696 - mse: 0.0696 - val_loss: 0.1033 - val_mse: 0.1033
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0657 - mse: 0.0657 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0539 - mse: 0.0539 - val_loss: 0.0973 - val_mse: 0.0973
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0414 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.1020 - val_mse: 0.1020
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0957 - val_mse: 0.0957
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0947 - val_mse: 0.0947
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.0922 - val_mse: 0.0922
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0984 - val_mse: 0.0984
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0017 - mse: 0.0017


[0.0017291202675551176, 0.0017291202675551176]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0980 - mse: 0.0980


[0.09803889691829681, 0.09803889691829681]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [25]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.3738 - mse: 0.3738 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1548 - mse: 0.1548 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1105 - mse: 0.1105 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0752 - mse: 0.0752 - val_loss: 0.1232 - val_mse: 0.1232
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0563 - mse: 0.0563 - val_loss: 0.1147 - val_mse: 0.1147
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0421 - mse: 0.0421 - val_loss: 0.1107 - val_mse: 0.1107
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0350 - mse: 0.0350 - val_loss: 0.1151 - val_mse: 0.1151
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0264 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0150 - mse: 0.0150 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.0974 - val_mse: 0.0974
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0028 - mse: 0

Epoch 123/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0878 - val_mse: 0.0878
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 9.9810e-04 - mse: 9.9810e-04 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.0893 - val_mse: 0.0893
Epoch 130/150
33/33 [==============================] - 0s 2ms

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0010 - mse: 0.0010


[0.0010358482832089067, 0.0010358482832089067]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0968 - mse: 0.0968


[0.09682624042034149, 0.09682624042034149]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [28]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1411 - mse: 0.1411


[0.14106802642345428, 0.14106802642345428]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [29]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29515.202849519173

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.